1.먼저 F 빼고 다 지웁시다.

In [66]:
import json
import os

- data 디렉토리는 설정해주세요. 저같은 경우는 ../data/에 있습니다.

In [26]:
file_dir = "../data/" # setting

- dir 세팅

In [32]:
step1 = os.listdir(file_dir)
step1

['CROWD_REAL005', 'CROWD_REAL006', 'CROWD_REAL004']

- 차례차례 들어가볼게요

In [65]:
for dir1 in step1:
    step2 = os.listdir(file_dir + dir1)
    #print(step2)
    for dir2 in step2:
        step3 = os.listdir(file_dir + dir1 + '/' + dir2)
        #print(step3)
        for dir3 in step3:
            file_list = os.listdir(file_dir + dir1 + '/' + dir2 + '/' + dir3)
            ## 여기 file list 파싱해서 F 다 지웁시다.
            for file in file_list:
                file_name = file_dir + dir1 + '/' + dir2 + '/' + dir3 + '/' + file
                if file.split('.')[0].split('_')[4] != 'F': # 삭제
                    os.remove(file_name)
#                     print(file_name) # 지워진 파일 출력